In [1]:
import pandas as pd
import numpy as np
import os
from IPython.display import display, Markdown
import plotly.express as px
import plotly.graph_objects as go
from google.colab import files


In [2]:
# Try loading local file first
filename_local = "Global_Superstore_synthetic.csv"

if os.path.exists(filename_local):
    df = pd.read_csv(filename_local)
    print(f"Loaded local file: {filename_local}")
else:
    print("Upload your CSV file now (Global_Superstore_synthetic.csv or cleaned CSV)")
    uploaded = files.upload()
    filename = next(iter(uploaded))
    df = pd.read_csv(filename)
    print(f"Loaded file: {filename}")

display(Markdown("### Raw Data Preview"))
df.head()


Upload your CSV file now (Global_Superstore_synthetic.csv or cleaned CSV)


Saving Global_Superstore_synthetic.csv to Global_Superstore_synthetic.csv
Loaded file: Global_Superstore_synthetic.csv


### Raw Data Preview

,Order ID,Order Date,Ship Date,Customer Name,Segment,Country,City,State,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,ORD-100000,2016-01-31 05:14:27,2016-02-13 05:14:27,Customer 1313,Consumer,United States,New York,GA,Central,PROD-6109,Office Supplies,Binders,Binders Model 57,940.93,5,0.15,274.03
1,ORD-100001,2015-10-19 21:17:16,2015-10-26 21:17:16,Customer 1263,Corporate,United States,Houston,NY,Central,PROD-7124,Technology,Computers,Computers Model 38,252.56,3,0.05,70.49
2,ORD-100002,2017-09-13 08:42:54,2017-09-26 08:42:54,Customer 133,Consumer,United States,Los Angeles,WA,West,PROD-2912,Office Supplies,Appliances,Appliances Model 126,140.85,6,0.00,-2.88
3,ORD-100003,2015-10-21 08:38:02,2015-11-03 08:38:02,Customer 999,Consumer,United States,Dallas,NY,West,PROD-4482,Office Supplies,Storage,Storage Model 87,153.32,4,0.00,-10.91
4,ORD-100004,2014-09-02 00:03:08,2014-09-04 00:03:08,Customer 786,Corporate,United States,Boston,WA,East,PROD-2531,Office Supplies,Binders,Binders Model 471,890.01,2,0.00,-43.85


In [4]:
def clean_superstore(df):
    df = df.copy()

    # Basic cleaning
    df.drop_duplicates(inplace=True)

    # Date formatting
    for col in ['Order Date', 'Ship Date']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Numeric conversions
    df['Sales'] = pd.to_numeric(df['Sales'], errors='coerce').fillna(0)
    df['Profit'] = pd.to_numeric(df['Profit'], errors='coerce').fillna(0)
    df['Quantity'] = pd.to_numeric(df.get('Quantity', 1), errors='coerce').fillna(1).astype(int)

    # Profit Margin
    df['Profit Margin'] = np.where(df['Sales'] != 0, df['Profit'] / df['Sales'], 0)

    # Date features
    df['Month'] = df['Order Date'].dt.month
    df['Year'] = df['Order Date'].dt.year
    df['MonthName'] = df['Order Date'].dt.strftime('%b')

    # Fill missing categories
    for col in ['Category', 'Region', 'Sub-Category', 'Customer Name']:
        if col in df.columns:
            df[col] = df[col].fillna("Unknown")

    return df


In [5]:
df = clean_superstore(df)

display(Markdown("### Cleaned Data Preview"))
df.head()


### Cleaned Data Preview

,Order ID,Order Date,Ship Date,Customer Name,Segment,Country,City,State,Region,Product ID,...,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Profit Margin,Month,Year,MonthName
0,ORD-100000,2016-01-31 05:14:27,2016-02-13 05:14:27,Customer 1313,Consumer,United States,New York,GA,Central,PROD-6109,...,Binders,Binders Model 57,940.93,5,0.15,274.03,0.291233,1,2016,Jan
1,ORD-100001,2015-10-19 21:17:16,2015-10-26 21:17:16,Customer 1263,Corporate,United States,Houston,NY,Central,PROD-7124,...,Computers,Computers Model 38,252.56,3,0.05,70.49,0.279102,10,2015,Oct
2,ORD-100002,2017-09-13 08:42:54,2017-09-26 08:42:54,Customer 133,Consumer,United States,Los Angeles,WA,West,PROD-2912,...,Appliances,Appliances Model 126,140.85,6,0.00,-2.88,-0.020447,9,2017,Sep
3,ORD-100003,2015-10-21 08:38:02,2015-11-03 08:38:02,Customer 999,Consumer,United States,Dallas,NY,West,PROD-4482,...,Storage,Storage Model 87,153.32,4,0.00,-10.91,-0.071158,10,2015,Oct
4,ORD-100004,2014-09-02 00:03:08,2014-09-04 00:03:08,Customer 786,Corporate,United States,Boston,WA,East,PROD-2531,...,Binders,Binders Model 471,890.01,2,0.00,-43.85,-0.049269,9,2014,Sep


In [6]:
total_sales = df['Sales'].sum()
total_profit = df['Profit'].sum()
avg_profit_margin = round(df['Profit Margin'].mean() * 100, 2)
num_orders = df['Order ID'].nunique() if 'Order ID' in df.columns else len(df)
num_customers = df['Customer Name'].nunique()

display(Markdown(f"""
## 📌 Key Business KPIs
- **Total Sales:** ${total_sales:,.2f}
- **Total Profit:** ${total_profit:,.2f}
- **Average Profit Margin:** {avg_profit_margin}%
- **Total Orders:** {num_orders:,}
- **Unique Customers:** {num_customers:,}
"""))



## 📌 Key Business KPIs  
- **Total Sales:** $1,772,162.99  
- **Total Profit:** $120,991.85  
- **Average Profit Margin:** 6.42%  
- **Total Orders:** 5,000  
- **Unique Customers:** 1,445  


In [7]:
sales_by_category = df.groupby("Category")[["Sales", "Profit"]].sum().reset_index().sort_values("Sales", ascending=False)
sales_by_region = df.groupby("Region")[["Sales", "Profit"]].sum().reset_index().sort_values("Sales", ascending=False)
monthly_sales = df.groupby(["Year", "Month"])[["Sales", "Profit"]].sum().reset_index()
top_products = df.groupby("Product Name")[["Sales", "Profit"]].sum().reset_index().sort_values("Sales", ascending=False).head(10)
top_customers = df.groupby("Customer Name")[["Sales", "Profit"]].sum().reset_index().sort_values("Sales", ascending=False).head(10)

display(Markdown("### 🔝 Top 10 Products"))
display(top_products)

display(Markdown("### 🔝 Top 10 Customers"))
display(top_customers)


### 🔝 Top 10 Products

,Product Name,Sales,Profit
583,Binders Model 140,3065.57,347.41
1965,Labels Model 167,2896.67,-178.01
488,Appliances Model 452,2750.16,544.81
2461,Paper Model 100,2666.90,14.75
1521,Copiers Model 165,2643.05,164.86
2588,Paper Model 296,2617.96,-181.06
1549,Copiers Model 227,2583.34,-40.00
818,Binders Model 57,2577.41,295.61
3378,Tables Model 282,2516.67,-129.28
2965,Phones Model 494,2473.37,-155.72


### 🔝 Top 10 Customers

,Customer Name,Sales,Profit
786,Customer 387,4559.95,252.11
110,Customer 1101,4285.31,93.70
970,Customer 560,4061.64,483.95
1282,Customer 849,3968.14,183.23
780,Customer 381,3922.99,108.38
1354,Customer 916,3718.95,561.80
120,Customer 1110,3665.91,-53.13
41,Customer 1036,3653.92,310.77
935,Customer 528,3643.03,526.29
622,Customer 232,3625.11,316.03


In [8]:
fig1 = px.bar(
    sales_by_category,
    x="Category",
    y="Sales",
    text_auto=".2s",
    title="Sales by Category",
)
fig1.show()


In [9]:
fig2 = px.bar(
    sales_by_region,
    x="Region",
    y="Profit",
    text_auto=".2s",
    title="Profit by Region",
)
fig2.show()


In [10]:
monthly_sales['MonthYear'] = monthly_sales['Year'].astype(str) + "-" + monthly_sales['Month'].astype(str).str.zfill(2)

fig3 = px.line(
    monthly_sales,
    x="MonthYear",
    y="Sales",
    markers=True,
    title="Monthly Sales Trend",
)
fig3.update_layout(xaxis_tickangle=-45)
fig3.show()


In [11]:
insights = []

# Insight 1 — Top category
top_cat = sales_by_category.iloc[0]
insights.append(f"Most profitable category is **{top_cat['Category']}** with ${top_cat['Profit']:.2f} profit.")

# Insight 2 — Best region
top_reg = sales_by_region.iloc[0]
insights.append(f"Highest sales region is **{top_reg['Region']}** with ${top_reg['Sales']:.2f} sales.")

# Insight 3 — Peak month
peak = monthly_sales.sort_values("Sales", ascending=False).iloc[0]
insights.append(f"Peak month is **{peak['Month']}/{peak['Year']}** with ${peak['Sales']:.2f} sales.")

display(Markdown("## ✨ Executive Summary"))
for i in insights:
    display(Markdown(f"- {i}"))


## ✨ Executive Summary

- Most profitable category is **Office Supplies** with $58609.18 profit.

- Highest sales region is **West** with $592400.93 sales.

- Peak month is **9/2015** with $46866.60 sales.

In [12]:
df.to_csv("Cleaned_Superstore_Data.csv", index=False)
files.download("Cleaned_Superstore_Data.csv")

with pd.ExcelWriter("Superstore_Summary.xlsx") as writer:
    df.to_excel(writer, sheet_name="cleaned_data", index=False)
    sales_by_category.to_excel(writer, sheet_name="sales_by_category", index=False)
    sales_by_region.to_excel(writer, sheet_name="sales_by_region", index=False)
    top_products.to_excel(writer, sheet_name="top_products", index=False)

files.download("Superstore_Summary.xlsx")

print("Files exported successfully!")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Files exported successfully!
